In [7]:
!pip install deims
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [8]:
import uuid
import json
import xmltodict
import deims
import datetime
import os

In [37]:
with open('getcapabilities_1.3.0.xml') as fd:
    doc = xmltodict.parse(fd.read())

for layer in doc['WMS_Capabilities']['Capability']['Layer']['Layer']:

    # date stamp of metadata update date
    now = datetime.datetime.now()

    # fetch EPSG code
    epsg_code = ''
    for value in layer['CRS']:
        if "EPSG:" in value:
            epsg_code = value[5:]

    # boundaries
    westBoundLongitude = float(layer['EX_GeographicBoundingBox']["westBoundLongitude"])
    eastBoundLongitude = float(layer['EX_GeographicBoundingBox']["eastBoundLongitude"])
    southBoundLatitude = float(layer['EX_GeographicBoundingBox']["southBoundLatitude"])
    northBoundLatitude = float(layer['EX_GeographicBoundingBox']["northBoundLatitude"])

    longitude_centre = (westBoundLongitude+eastBoundLongitude)/2.0
    latitude_centre = (northBoundLatitude+southBoundLatitude)/2.0

    keywords_to_be_printed = []
    record_uuid = None
    authors = None
    so_reference = None
    # process keywords for site information and temporal extent
    for keyword in layer['KeywordList']['Keyword']:
        uri = None

        if "id: " in keyword:
            record_uuid = keyword[3:]
            continue

        if "project: " in keyword:
            if "e-shape" in keyword:
                project = {
                    "name": "e-shape",
                    "PID": "https://doi.org/10.3030/820852"
                }
                continue

            if "AGAME" in keyword:
                project = {
                    "name": "AGAME",
                    "PID": "https://www.umweltbundesamt.at/en/services/agame"
                }
                continue

        if "variable: " in keyword:
            data_product_type = keyword[10:]
            if data_product_type == "Phenology":
                keyword = "Phenology"
                uri = "https://vocabs.lter-europe.net/EnvThes/21647"
                authors = [{
                  "fullName": "Saverio Vicario",
                  "givenName": "Saverio",
                  "familyName": "Vicario",
                  "email": "saverio.vicario@iia.cnr.it",
                  "type": "ContactPerson",
                  "ids": [{
                      "id": "https://orcid.org/0000-0003-1140-0483",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "Hydroperiod":
                keyword = "Hydroperiod"
                authors = [{
                  "fullName": "Ioannis Manakos",
                  "givenName": "Ioannis",
                  "familyName": "Manakos",
                  "email": "imanakos@iti.gr",
                  "type": "ContactPerson",
                  "ids": [{
                      "id": "https://orcid.org/0000-0001-6833-294X",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "GPP":
                keyword = "GPP"
                uri = "http://vocabs.lter-europe.net/EnvThes/21074"
                so_reference = {
                    "name": "Gross Primary Production",
                    "url": "http://vocabs.lter-europe.net/EnvThes/21074"
                },
                authors = [{
                  "fullName": "Mario Alberto Fruentes-Monjaraz",
                  "givenName": "Alberto",
                  "familyName": "Fuentes-Monjaraz",
                  "type": "ContactPerson",
                  "email": "mario.fuentesmonjaraz@deltares.nl",
                  "ids": [{
                      "id": "https://orcid.org/0000-0002-0711-8867",
                      "schema": "Orcid",
                    }]
                },
                {
                  "fullName": "Valeria Mobilia",
                  "givenName": "Valeria",
                  "familyName": "Mobilia",
                  "type": "ContactPerson",
                  "email": "valeria.mobilia@deltares.nl",
                  "ids": [{
                      "id": "https://orcid.org/0000-0001-9370-598X",
                      "schema": "Orcid",
                    }]
                }]

            if data_product_type == "snowcover":
                uri = "https://vocabs.lter-europe.net/EnvThes/21559"
                keyword = "snowCover"
                authors = [{
                  "fullName": "Maria Adamo",
                  "givenName": "Maria",
                  "familyName": "Adamo",
                  "type": "ContactPerson",
                  "email": "adamo@iia.cnr.it",
                  "ids": [{
                      "id": "https://orcid.org/0000-0003-3030-4884",
                      "schema": "Orcid",
                    }]
                },
                {
                  "fullName": "Chiara Richiardi",
                  "givenName": "Chiara",
                  "familyName": "Richiardi",
                  "type": "ContactPerson",
                  "email": "chiara.richiardi@iia.cnr.it",
                  "ids": [{
                      "id": "https://orcid.org/0000-0002-2370-7768",
                      "schema": "Orcid",
                    }]
                }]

        if "site: " in keyword:
            deims_suffix = keyword[6:]
            #print(site_record)
            uri = "https://deims.org/" + deims_suffix
            site_record = deims.getSiteById(deims_suffix)

            try:
              site_name = site_record["title"]
            except:
              print(layer['Title'])
              print(deims_suffix)
            continue

        if "https://doi.org/" in keyword:
            doi = {
                "name": "B2Share",
                "description": "Download data on B2Share",
                "url": keyword
            }

        if "time: " in keyword:
            time_string = keyword[6:]

            if "-" in time_string:
                time_values = time_string.split('-')
                period = [{
                    "startDate": time_values[0] + '-01-01',
                    "endDate": time_values[1] + '-12-31'
                }]

            else:
                period = [{
                    "startDate": time_string + '-01-01',
                    "endDate": time_string + '-12-31'
                }]
            continue

        if uri:
            formatted_keyword = {
                "name": keyword,
                "url": uri
            }
        else:
            formatted_keyword = {
                "name": keyword,
                "url": ""
            }

        keywords_to_be_printed.append(formatted_keyword)

    #print(deims_suffix)

    record = {
        "metadata": {
            "datasetIds": [
                {
                    "source": "eLTER spatial.io",
                    "identifier": record_uuid,
                    "type": "PID"
                }
            ],
            "titles": [{
                "language": "eng",
                "text": layer['Title']
            }],
            "descriptions": [{
                "type": "Abstract",
                "language": "eng",
                "description": layer['Abstract'] # we could add a sentence that the metadata record was generated automatically
            }],
            "keywords": keywords_to_be_printed,
            "publicationDate": datetime.datetime.now().strftime('%Y-%m-%d'),
            "language": "english",
            "SOReference": so_reference,
            "authors": authors,
            "geoLocations": [
                {
                    "EX_GeographicDescription": site_record["title"],
                    "Point": {
                        "longitude": longitude_centre,
                        "latitude": latitude_centre
                    }
                },
                {
                      "EX_GeographicDescription": site_record["title"],
                      "EX_GeographicBoundingBox": {
                           "westLongitude": westBoundLongitude,
                           "eastLongitude": eastBoundLongitude,
                           "southLatitude": southBoundLatitude,
                           "northLatitude": northBoundLatitude 
                      }
                 }
            ],
            "temporalCoverages": period,
            "geoServerInfo": {
                "service": {
                    "type": "WMS"
                },
                "mapDataDefinition": {
                    "mapData": [
                        {
                            "path": "https://spatialnode.elter.cerit-sc.cz/geoserver/ows?version=1.3.0",
                            "epsgCode": epsg_code,
                            "type": "POLYGON",
                            "features": {
                                "name": layer['Name'],
                                "label": layer['Title'],
                                "style": {
                                    "colour": "#000000"
                                }
                            },
                            "bytetype": False
                      }
                    ]
                }
            },
            "project": project,
            "siteReference": [{
                "name": site_record["title"],
                "PID": "https://deims.org/" + deims_suffix
            }],
        }
    }
    
    filename = record_uuid + ".json"
    base_dir = os.getcwd() + '/files_for_catalogue'
    destination_path = os.path.join(base_dir, filename)

    with open(destination_path, "w+") as f:
        json.dump(record, f)


KeyboardInterrupt: 